# script for finding buildings with specific attribute values pairs

In [1]:
import pandas as pd
from pathlib import Path
import os
from buildings_bench.data.buildings900K import Buildings900K
import random
import numpy as np

In [2]:
metadata_path = Path(os.environ.get('BUILDINGS_BENCH', ''), 'metadata_dev')

df1 = pd.read_parquet(metadata_path / "comstock_amy2018.parquet", engine="pyarrow")
df2 = pd.read_parquet(metadata_path / "comstock_tmy3.parquet", engine="pyarrow")

In [3]:
com_chars = np.array([
   "in.building_subtype",
    "in.building_type",
    "in.number_of_stories",
    "in.sqft",
    "in.hvac_system_type",
    "in.weekday_operating_hours",
    "in.weekday_opening_time",
    "in.weekend_operating_hours",
    "in.weekend_opening_time",
    "in.tstat_clg_delta_f",
    "in.tstat_clg_sp_f",
    "in.tstat_htg_delta_f",
    "in.tstat_htg_sp_f"
])

In [4]:
bldg_ids = set()
metadata_path = Path(os.environ.get('BUILDINGS_BENCH', ''), 'metadata_dev')
with open(metadata_path / "comcap_350k_remain_new.idx") as f:
    for i, line in enumerate(f):
        bldg_id = int(line.split("\t")[-2])
        bldg_ids.add(bldg_id)
print(len(bldg_ids))

336817


In [5]:
df2 = df2[df2.index.isin(bldg_ids)]
df1 = df1[df1.index.isin(bldg_ids) & ~df1.index.isin(df2.index)]

In [6]:
df = pd.concat([df1, df2])
df

,applicability,in.upgrade_name,in.tstat_clg_delta_f,in.tstat_clg_sp_f,in.tstat_htg_delta_f,in.tstat_htg_sp_f,in.aspect_ratio,in.building_subtype,in.county,in.building_type,...,out.electricity.total.energy_consumption_intensity,out.site_energy.total.energy_consumption,out.site_energy.total.energy_consumption_intensity,out.natural_gas.total.energy_consumption,out.natural_gas.total.energy_consumption_intensity,out.other_fuel.total.energy_consumption,out.other_fuel.total.energy_consumption_intensity,upgrade,weight,metadata_index
bldg_id,,,,,,,,,,,,,,,,,,,,,
51065,True,Baseline,0.0,75.0,5.0,69.0,1.0,None,G0101010,SmallOffice,...,10.874603,2.003222e+05,11.446984,10014.271788,0.572244,0.0,0.0,00,7.041741,775
54245,True,Baseline,0.0,73.0,0.0,66.0,1.0,None,G0101170,SmallOffice,...,11.356190,1.987333e+05,11.356190,0.000000,0.000000,0.0,0.0,00,7.041741,826
57306,True,Baseline,0.0,74.0,0.0,68.0,1.0,None,G0100830,SmallOffice,...,19.786111,6.311944e+04,21.039815,3760.211869,1.253404,0.0,0.0,00,7.041741,868
59474,True,Baseline,10.0,70.0,7.0,68.0,2.0,strip_mall_restaurant10,G0100970,RetailStripmall,...,19.954667,1.936081e+06,25.814407,439375.480434,5.858340,0.0,0.0,00,1.947971,901
70885,True,Baseline,0.0,72.0,0.0,68.0,1.0,None,G0101130,RetailStandalone,...,17.301481,1.682833e+05,22.437778,38510.234855,5.134698,0.0,0.0,00,3.030624,1070
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345431,True,Baseline,3.0,75.0,3.0,65.0,1.0,None,G5600390,SmallOffice,...,7.767143,2.217139e+05,12.669365,85768.377679,4.901050,0.0,0.0,00,7.041741,345595
347436,True,Baseline,10.0,71.0,8.0,68.0,2.0,None,G5600210,PrimarySchool,...,7.658222,5.636611e+05,15.030963,276411.674885,7.370978,0.0,0.0,00,11.467280,345596
348460,True,Baseline,10.0,72.0,6.0,68.0,1.0,None,G5600130,RetailStandalone,...,9.962000,6.436278e+05,17.163407,269985.433921,7.199612,0.0,0.0,00,3.030624,345597


In [7]:
df.to_csv(metadata_path / "AttributeCaps" / "comstock_340k.csv")

In [33]:
com_chars = [
    "in.building_subtype",
    "in.building_type",
    "in.number_of_stories",
    "in.sqft",
    "in.hvac_system_type",
    "in.weekday_operating_hours",
    "in.weekday_opening_time",
    "in.weekend_operating_hours",
    "in.weekend_opening_time",
    # "in.tstat_clg_delta_f",
    # "in.tstat_clg_sp_f",
    # "in.tstat_htg_delta_f",
    # "in.tstat_htg_sp_f"
]

# randomly split
def ood_split(df, feature_names):
    # randomly choose a feature
    features = np.random.choice(feature_names, size=2, replace=False)
    
    values = []
    for ft in features:
        # randomly choose a value
        val = np.random.choice(df[ft].unique())
        values.append(val)
        
    print("selected features", features)
    print("selected values", values)
    
    mask = np.ones(len(df), dtype=bool)
    for ft, val in zip(features, values):
        mask = np.logical_and(mask, df[ft].values == val)
        
    print(mask)
    print(np.sum(mask))

    
ood_split(df, com_chars)

selected features ['in.number_of_stories' 'in.building_subtype']
selected values [7.0, 'mediumoffice_datacenter']
[False False False ... False False False]
41
